# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [17]:

from utils import get_dataset
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import tensorflow as tf


%matplotlib inline

In [ ]:
dataset = get_dataset("dataset/*.tfrecord")

## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [ ]:
def display_images(batch):
    # TODO - Your implementation here
    import matplotlib as mpl
    mpl.use("TkAgg")
    fig, ax = plt.subplots(2,5,figsize = (20,10))
    for idx, data in enumerate(batch):
        if idx == 10:
            break
        img = data["image"].numpy()
        ax[int(idx/5), idx%5].imshow(img)
        BoxAndClass = zip(data["groundtruth_boxes"], data["groundtruth_classes"])
        for box, cl in BoxAndClass:
            box = box.numpy()*640
            cl = tf.squeeze(cl)
            y1, x1 = box[:2]
            y2, x2 = box[2:4]
            if cl == 1:
                color = 'red'
            elif cl == 2:
                color = 'blue'
            elif cl == 4:
                color = 'green'
            rect = patches.Rectangle((x1,y1),x2-x1,y2-y1, linewidth = 2, edgecolor  = color, fill= False)
            ax[int(idx/5), idx%5].add_patch(rect)
    fig.tight_layout(pad = 0.5)
    plt.show()

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
    
# Display 10 random images in dataset
dataset = dataset.shuffle(buffer_size = 100)
display_images(dataset)






## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...